In [2]:
import pandas as pd
import numpy as np
import scripts.data_pull as dp
import scripts.preprocess_text as ppt
import scripts.blosc_interface as bi

https://stackoverflow.com/questions/26826002/adding-words-to-stop-words-list-in-tfidfvectorizer-in-sklearn
https://stackoverflow.com/questions/50756085/how-to-print-the-progress-of-a-list-comprehension-in-python
https://stackoverflow.com/questions/57983431/whats-the-most-space-efficient-way-to-compress-serialized-python-data

In [3]:
tokenized_corpus = bi.blosc_read("./data/tokenized_corpus.dat")

In [4]:
tokenized_corpus.head()

,art_title,art_author,art_date,art_topic,art_link,art_source,text,art_bias,stemmed_tokens,text_tokens,tokens,stems,text_lem
0,The emerging markets e-commerce opportunity,Christoph Ungerer,2021-03-26,future-development,https://www.brookings.edu/blog/future-developm...,Brookings Institute,while e-commerce giants such as amazon dominat...,left-wing,"[e-commerc, giant, amazon, domin, headlin, cov...","[e-commerce, giants, amazon, dominate, headlin...","e-commerce,giants,amazon,dominate,headlines,co...","e-commerc,giant,amazon,domin,headlin,covid-19,...","e-commerce,giants,amazon,dominate,headlines,co..."
1,"""1619"" Pulitzer Will Boost Socialist Teaching ...",Mike Gonzalez,2020-05-11,education,https://www.heritage.org/education/commentary/...,Heritage Commentary,the pulitzer prize board this week awarded its...,right-wing,"[pulitz, prize, board, week, award, commentari...","[pulitzer, prize, board, week, awarded, commen...","pulitzer,prize,board,week,awarded,commentary,a...","pulitz,prize,board,week,award,commentari,award...","pulitzer,prize,board,week,awarded,commentary,a..."
2,"""1983"" and ""The Brink"" Review: The Most Danger...","Lee Edwards, Ph.D.",2018-08-10,defense,https://www.heritage.org/defense/commentary/19...,Heritage Commentary,most historians agree that the world came clos...,right-wing,"[historian, agre, world, came, closest, nuclea...","[historians, agree, world, came, closest, nucl...","historians,agree,world,came,closest,nuclear,wa...","historian,agre,world,came,closest,nuclear,war,...","historians,agree,world,came,closest,nuclear,wa..."
3,"""60 Minutes"" Snubs the Facts on Education",Jonathan Butcher,2018-03-13,education,https://www.heritage.org/education/commentary/...,Heritage Commentary,beth richardson is committed to her son’s succ...,right-wing,"[beth, richardson, commit, son, success, expec...","[beth, richardson, committed, son, success, ex...","beth,richardson,committed,son,success,expected...","beth,richardson,commit,son,success,expect,jed,...","beth,richardson,committed,son,success,expected..."
4,"""As Israel and the Arabs Battle, Moscow Collec...",James Phillips,1983-09-20,europe,https://www.heritage.org/europe/report/israel-...,Heritage Report,"i i 291 september 20, 1983 as israel and the a...",right-wing,"[septemb, israel, arab, ban4, moscow, collect,...","[september, israel, arabs, ban4, moscow, colle...","september,israel,arabs,ban4,moscow,collects,di...","septemb,israel,arab,ban4,moscow,collect,divide...","september,israel,arabs,ban4,moscow,collects,di..."


In [22]:
count_vect = bi.blosc_read("./data/count_vect.dat")
tfidf_vect = bi.blosc_read("./data/tfidf_vect.dat")

In [23]:
count_vect.get_feature_names_out()

array(['$', "''", "'s", '(', ')', ',', '.', ':', ';', '?', '[', ']', '``',
       'act', 'action', 'administr', 'allow', 'also', 'america',
       'american', 'among', 'author', 'back', 'base', 'becom', 'benefit',
       'biden', 'bill', 'billion', 'budget', 'busi', 'call', 'campaign',
       'capit', 'care', 'case', 'chang', 'china', 'citi', 'come',
       'commun', 'compani', 'congress', 'constitut', 'continu', 'control',
       'cost', 'could', 'countri', 'court', 'creat', 'current', 'day',
       'defens', 'democrat', 'develop', 'differ', 'econom', 'economi',
       'educ', 'effect', 'effort', 'elect', 'employ', 'end', 'even',
       'exampl', 'fact', 'famili', 'feder', 'first', 'forc', 'fund',
       'gener', 'get', 'go', 'good', 'govern', 'group', 'health', 'help',
       'hous', 'howev', 'human', 'import', 'includ', 'incom', 'increas',
       'individu', 'industri', 'institut', 'interest', 'intern', 'invest',
       'issu', 'job', 'labor', 'last', 'law', 'left', 'less', 'level',

In [24]:
x_train = bi.blosc_read("./data/corpus_tfidf_vectorized_train.dat")

x_test = bi.blosc_read("./data/corpus_tfidf_vectorized_test.dat")

In [25]:
y_train = bi.blosc_read("./data/corpus_bias_train.dat")

y_test = bi.blosc_read("./data/corpus_bias_test.dat")

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_vect = TfidfTransformer()

x_train_tfidf = tfidf_vect.fit_transform(x_train)
x_test_tfidf = tfidf_vect.fit_transform(x_test)

In [26]:
from sklearn.linear_model import LogisticRegression

log_reg_clf = LogisticRegression()

In [27]:
log_reg_clf.fit(x_train, y_train)

LogisticRegression()

In [28]:
predicted = log_reg_clf.predict(x_test)
np.mean(predicted == y_test)

0.8168658088235294

In [29]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

   left-wing       0.83      0.78      0.80      2097
  right-wing       0.80      0.86      0.83      2255

    accuracy                           0.82      4352
   macro avg       0.82      0.82      0.82      4352
weighted avg       0.82      0.82      0.82      4352



In [10]:
metrics.confusion_matrix(y_test, predicted)

array([[1656,  441],
       [ 273, 1982]])

In [30]:
coef_values = pd.DataFrame(log_reg_clf.coef_, columns=count_vect.get_feature_names_out())

In [31]:
coef_values.head()

,$,'','s,(,),",",.,:,;,?,...,without,work,worker,world,would,year,—,’,“,”
0,0.536189,4.448389,8.424073,-2.549232,-2.394622,-1.783803,2.707115,-1.196809,-1.617017,1.425718,...,0.370157,-2.502709,-3.537142,-0.004656,2.65586,-0.102624,-15.382753,-1.495565,1.457116,2.238521


In [15]:
tfidf_vect.get_params()

{'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': False, 'use_idf': True}

In [32]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)

sgd_clf.fit(x_train, y_train)

SGDClassifier(alpha=0.001, max_iter=5, random_state=42, tol=None)

In [33]:
sgd_predict = sgd_clf.predict(x_test)
np.mean(sgd_predict == y_test)

0.8295036764705882

In [34]:
print(metrics.classification_report(y_test, sgd_predict))

              precision    recall  f1-score   support

   left-wing       0.84      0.80      0.82      2097
  right-wing       0.82      0.85      0.84      2255

    accuracy                           0.83      4352
   macro avg       0.83      0.83      0.83      4352
weighted avg       0.83      0.83      0.83      4352



In [36]:
metrics.confusion_matrix(y_test, sgd_predict)

array([[1688,  409],
       [ 333, 1922]])

In [37]:
from sklearn import tree

tree_clf = tree.DecisionTreeClassifier()

tree_clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [38]:
tree_predict = tree_clf.predict(x_test)
np.mean(tree_predict == y_test)

0.7842371323529411

In [52]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(alpha=1e-5, random_state=1, hidden_layer_sizes=(100, 10), solver="sgd", max_iter=1000)

In [53]:
nn_clf.fit(x_train, y_train)

/home/keenans/miniconda3/envs/political_sentiment/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 10), max_iter=1000,
              random_state=1, solver='sgd')

In [54]:
nn_predict = nn_clf.predict(x_test)
np.mean(nn_predict == y_test)

0.8150275735294118